# Experimentacion: Modelos

En este notebok se experimenta con los distintos modelos propuestos en el proyecto y se exploran modelos adicionales

## Referencias

1) Clase del profesor alfredo Garbuno
2) Documentacion de cmdpystan

In [1]:
# import all libraries used in this notebook
import os
import numpy as np
import pandas as pd
from cmdstanpy import CmdStanModel
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from IPython.core.magic import register_cell_magic
import time
from outils import load_config, log_run

In [2]:
from statistics import mean

In [3]:
# Load config file calling load_config function
config_f = load_config("config.yaml")
LOGFILE = config_f["log_file"]

### Datos

In [4]:
radon = pd.read_csv(os.path.join(config_f["data_directory"],"radon.csv"))

In [5]:
radon.head(2)

,state,floor,county,log_radon,log_uranium
0,AZ,1,APACHE,-1.203973,0.817297
1,AZ,9,APACHE,-0.510826,0.817297


Restringimos los datos al estado de Minesota para guardar consistencia con el proyecto

In [6]:
mn_radon = radon[radon['state']=='MN'].reset_index(drop=True)
mn_radon.drop(columns=['state'], inplace=True)

In [7]:
mn_radon['county_id'] = pd.factorize(mn_radon['county'])[0]+ 1

In [8]:
mn_radon

,floor,county,log_radon,log_uranium,county_id
0,1,AITKIN,0.788457,-0.689048,1
1,0,AITKIN,0.788457,-0.689048,1
2,0,AITKIN,1.064711,-0.689048,1
3,0,AITKIN,0.000000,-0.689048,1
4,0,ANOKA,1.131402,-0.847313,2
...,...,...,...,...,...
922,0,WRIGHT,1.856298,-0.090024,84
923,0,WRIGHT,1.504077,-0.090024,84
924,0,WRIGHT,1.609438,-0.090024,84
925,0,YELLOW MEDICINE,1.308333,0.355287,85


### Diccionario de datos

In [9]:
radon_data = {"N": len(mn_radon), 
              "x": mn_radon.floor.astype(float), 
              "y": mn_radon.log_radon,
              "J":85, 
              "county" : mn_radon.county_id}

# Replicacion de experimentos sugeridos

En esta seccion replicamos los experimentos sugeridos en nuestro proyecto de referencia. Notar que usamos parametros dados por los autores originales del ejercicio, por lo cual las previas podrian ser no informativas y resultar en observaciones no usuales.

## Modelo 1: Regresion Lineal (no multinivel) coeficientes invariantes por condado

El primer modelo propuesto se denomina "complete pooling" esto debido a que se estiman los mismos coeficientes para todos los condados.

### Modelo agrupado

$ \large log\_radon_i  \sim N( \alpha + \beta Piso_i  , \sigma)$

$\large \alpha \sim N(\mu_{\alpha},\sigma_{\alpha})$ <br>
$\large \beta \sim N(\mu_{\beta},\sigma_{\beta})$ <br>
$\large \sigma \sim N(\mu_{\sigma},\sigma_{\sigma})$

Media de alpha: 1.33

### Hot intake

El modelo de intercepto no variable por estado (complete pooling) tiene como principal deficiencia el hecho de que no toma en cuenta las variaciones de que pueden existir entre condados, la cual es la principal motivacion del estudio

### Definimos el modelo en Stan

In [10]:
modelString="""
    data {
        int<lower=1> N;
        vector[N] x;
        vector[N] y;
        }
    parameters {
        real alpha;
        real beta;
        real<lower=0> sigma;
        }
    model {
        y ~ normal(alpha + beta * x, sigma);
        alpha ~ normal(0, 10);
        beta ~ normal(0, 10);
        sigma ~ normal(0, 10);
        }
    generated quantities {
        array[N] real y_rep = normal_rng(alpha + beta * x, sigma);
        }
        
"""

In [11]:
# Creo el archivo de Stan
modelo_agrupado = os.path.join(config_f["models_directory"],"modelo_agrupado.stan")
with open(modelo_agrupado, 'w') as f:
    f.write(modelString)

### Genero las cadenas

Esto producira 6 cadenas de 1,000 iteraciones cada una

In [13]:
%%log_run Modelo agrupado (regresion lineal)
modelo_agrupado = CmdStanModel(stan_file=os.path.join(config_f["models_directory"], "modelo_agrupado.stan"))
modelo_agrupado_fit = modelo_agrupado.sample(
    data=radon_data, 
    show_progress=False, 
    chains=6,
    iter_warmup= 1000,
    iter_sampling=10000)

22:24:37 - cmdstanpy - INFO - CmdStan start processing
22:24:37 - cmdstanpy - INFO - Chain [1] start processing
22:24:37 - cmdstanpy - INFO - Chain [2] start processing
22:24:37 - cmdstanpy - INFO - Chain [3] start processing
22:24:37 - cmdstanpy - INFO - Chain [4] start processing
22:24:37 - cmdstanpy - INFO - Chain [5] start processing
22:24:37 - cmdstanpy - INFO - Chain [6] start processing
22:24:43 - cmdstanpy - INFO - Chain [5] done processing
22:24:43 - cmdstanpy - INFO - Chain [4] done processing
22:24:45 - cmdstanpy - INFO - Chain [2] done processing
22:24:46 - cmdstanpy - INFO - Chain [6] done processing
22:24:46 - cmdstanpy - INFO - Chain [3] done processing
22:24:46 - cmdstanpy - INFO - Chain [1] done processing


<ExecutionResult object at 7f7507cec390, execution_count=None error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f7507cec510, raw_cell="modelo_agrupado = CmdStanModel(stan_file=os.path.j.." store_history=False silent=False shell_futures=True cell_id=None> result=None>

### Analizando las cadenas

In [15]:
agrupado_pd = modelo_agrupado_fit.draws_pd(vars=['alpha', 'beta', 'sigma'])
print(f'sample draws shape:  {agrupado_pd.shape}')
agrupado_pd

sample draws shape:  (60000, 3)


,alpha,beta,sigma
0,1.26026,-0.517698,0.839185
1,1.28302,-0.534180,0.825683
2,1.29139,-0.428177,0.818308
3,1.29458,-0.607829,0.868602
4,1.32707,-0.673792,0.839358
...,...,...,...
59995,1.35577,-0.608309,0.829686
59996,1.32997,-0.652155,0.820288
59997,1.34122,-0.602490,0.840932
59998,1.31826,-0.614990,0.810389


In [16]:
modelo_agrupado_fit.summary().round(2)

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
lp__,-295.12,0.01,1.24,-297.54,-294.80,-293.80,29215.0,615.09,1.0
alpha,1.33,0.00,0.03,1.28,1.33,1.38,43058.3,906.55,1.0
beta,-0.61,0.00,0.07,-0.73,-0.61,-0.49,42207.3,888.63,1.0
sigma,0.83,0.00,0.02,0.80,0.83,0.87,48197.8,1014.75,1.0
y_rep[1],0.73,0.00,0.84,-0.65,0.72,2.11,59469.7,1252.07,1.0
...,...,...,...,...,...,...,...,...,...
y_rep[923],1.33,0.00,0.84,-0.05,1.33,2.71,60417.5,1272.03,1.0
y_rep[924],1.33,0.00,0.84,-0.04,1.33,2.71,60125.3,1265.88,1.0
y_rep[925],1.33,0.00,0.84,-0.05,1.32,2.72,59302.8,1248.56,1.0
y_rep[926],1.32,0.00,0.84,-0.04,1.32,2.71,57773.2,1216.36,1.0


## Modelo 2: Regresion Lineal (no multinivel) intercepto variantes por condado

El segundo modelo se denomina no-pooling debido a que no agregan en un mismo coeficiente los efectos de los condados, se estiman por separado y se genera un coeficiente de intercepto para cada condado

### Modelo no agrupado

$ \large log\_radon_i  \sim N( \alpha_{j[i]} + \beta Piso_i  , \sigma)$

$\large \alpha_j \sim N(\mu_{\alpha},\sigma_{\alpha})$ <br>
$\large \beta \sim N(\mu_{\beta},\sigma_{\beta})$ <br>
$\large \sigma \sim N(\mu_{\sigma},\sigma_{\sigma})$

$\large j \in \{1,2....85\}$

Media de alpha: 1.50

Sd de alpha: 0.46

### Hot intake

El modelo con intercepto variable por condado tiene el problema de que cuando se encuentra con condados con pocas observaciones de casas, se hace un sobreajuste a esa cantidad baja de datos lo cual vuelve el parametro estimado poco confiable

In [17]:
modelString="""
    data {
        int<lower=1> N;  // observations
        int<lower=1> J;  // counties
        array[N] int<lower=1, upper=J> county;
        vector[N] x;     // floor
        vector[N] y;     // radon
        }
    parameters {
        vector[J] alpha;
        real beta;
        real<lower=0> sigma;
        }
    model {
        y ~ normal(alpha[county] + beta * x, sigma);  
        alpha ~ normal(0, 10);
        beta ~ normal(0, 10);
        sigma ~ normal(0, 10);
        }
    generated quantities {
        array[N] real y_rep = normal_rng(alpha[county] + beta * x, sigma);
        }

"""

In [18]:
# Creo el archivo de Stan
modelo_no_agrupado = os.path.join(config_f["models_directory"],"modelo_no_agrupado.stan")
with open(modelo_no_agrupado, 'w') as f:
    f.write(modelString)

In [19]:
%%log_run Modelo no agrupado (intercepto por condado)
modelo_no_agrupado = CmdStanModel(stan_file=os.path.join(config_f["models_directory"], "modelo_no_agrupado.stan"))
modelo_no_agrupado_fit = modelo_no_agrupado.sample(
    data=radon_data, 
    show_progress=False, 
    chains=6,
    iter_warmup= 1000,
    iter_sampling=10000)

22:36:53 - cmdstanpy - INFO - compiling stan file /home/juanpalms/Desktop/MCD/Bayesiana/PROYECTO/modelacion-bayesiana-proyecto-final-bayes-radioactivo/modelos/modelo_no_agrupado.stan to exe file /home/juanpalms/Desktop/MCD/Bayesiana/PROYECTO/modelacion-bayesiana-proyecto-final-bayes-radioactivo/modelos/modelo_no_agrupado
22:37:06 - cmdstanpy - INFO - compiled model executable: /home/juanpalms/Desktop/MCD/Bayesiana/PROYECTO/modelacion-bayesiana-proyecto-final-bayes-radioactivo/modelos/modelo_no_agrupado
22:37:06 - cmdstanpy - INFO - CmdStan start processing
22:37:06 - cmdstanpy - INFO - Chain [1] start processing
22:37:06 - cmdstanpy - INFO - Chain [2] start processing
22:37:06 - cmdstanpy - INFO - Chain [3] start processing
22:37:06 - cmdstanpy - INFO - Chain [4] start processing
22:37:06 - cmdstanpy - INFO - Chain [5] start processing
22:37:06 - cmdstanpy - INFO - Chain [6] start processing
22:37:14 - cmdstanpy - INFO - Chain [5] done processing
22:37:14 - cmdstanpy - INFO - Chain [2]

<ExecutionResult object at 7f7505f00d50, execution_count=None error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f7505f00610, raw_cell="modelo_no_agrupado = CmdStanModel(stan_file=os.pat.." store_history=False silent=False shell_futures=True cell_id=None> result=None>

### Analizando las cadenas

In [20]:
modelo_no_agrupado_pd = modelo_no_agrupado_fit.draws_pd(vars=['alpha', 'beta', 'sigma'])
print(f'sample draws shape:  {modelo_no_agrupado_pd.shape}')
modelo_no_agrupado_pd.head(3)

sample draws shape:  (60000, 87)


,alpha[1],alpha[2],alpha[3],alpha[4],alpha[5],alpha[6],alpha[7],alpha[8],alpha[9],alpha[10],...,alpha[78],alpha[79],alpha[80],alpha[81],alpha[82],alpha[83],alpha[84],alpha[85],beta,sigma
0,1.234910,0.821204,1.71428,1.34633,0.945321,1.01908,2.17994,2.979810,0.934876,1.45177,...,1.676570,0.263096,1.21551,3.49169,1.56604,1.49645,2.28122,0.928500,-0.552123,0.752007
1,1.305110,0.901274,2.32929,1.57724,2.203810,2.27955,2.02432,0.868734,0.917167,1.54948,...,0.946364,0.951878,1.28535,2.62326,3.34635,1.62007,1.15405,1.721150,-0.839822,0.803419
2,0.432238,0.899135,0.95534,1.66418,1.071570,1.13331,2.23239,2.561430,1.386400,1.62642,...,1.759830,0.280802,1.29419,2.83199,2.18772,1.81107,1.94520,0.882498,-0.681609,0.737269


In [21]:
summary=modelo_no_agrupado_fit.summary().round(2)

In [27]:
(summary[1:86]["R_hat"]>=1).count()

85

In [31]:
summary[1:86]["N_Eff"].min()

72189.1

## Modelo 3: modelo jerarquico intercepto variantes por condado (partial pooling)

En este modelo se toma en cuenta la jerarquia de los condados, entonces se asume que los condados vienen de una distribucion comun de condados.

El modelo jerarquico es una extension del modelo no-pooling (con interceptos variantes por condado), hay un intercepto unico para los condados que viene de una distribucion comun.

### Modelo parcialmente agrupado

$ \large log\_radon_i  \sim N( \alpha_{j[i]} + \beta Piso_i  , \sigma)$ <br>

$\large \alpha_j \sim N(\mu_{\alpha},\sigma_{\alpha})$ <br>
$\large \mu_\alpha \sim N(\mu,\sigma)$ <br>
$\large \sigma_\alpha \sim N(\mu,\sigma)$ <br>
$\large \beta \sim N(\mu_{\beta},\sigma_{\beta})$ <br>
$\large \sigma \sim N(\mu_{\sigma},\sigma_{\sigma})$ <br>

$\large j \in \{1,2....85\}$

Media de alpha: 1.47

Sd de alpha: 0.37

In [33]:
modelString="""
    data {
        int<lower=1> N;  // observations
        int<lower=1> J;  // counties
        array[N] int<lower=1, upper=J> county;
        vector[N] x;
        vector[N] y;
        }
   parameters {
       real mu_alpha;
       real<lower=0> sigma_alpha;
       vector<offset=mu_alpha, multiplier=sigma_alpha>[J] alpha;  // non-centered parameterization
       real beta;
       real<lower=0> sigma;
       }
    model {
        y ~ normal(alpha[county] + beta * x, sigma);  
        alpha ~ normal(mu_alpha, sigma_alpha); // partial-pooling
        beta ~ normal(0, 10);
        sigma ~ normal(0, 10);
        mu_alpha ~ normal(0, 10);
        sigma_alpha ~ normal(0, 10);
        }
    generated quantities {
        array[N] real y_rep = normal_rng(alpha[county] + beta * x, sigma);
        }

"""

In [34]:
# Creo el archivo de Stan
model_parcialmente_agrupado = os.path.join(config_f["models_directory"],"model_parcialmente_agrupado.stan")
with open(model_parcialmente_agrupado, 'w') as f:
    f.write(modelString)

In [35]:
%%log_run Modelo parcialmente agrupado (jerarquico solo con intercepto)
model_parcialmente_agrupado = CmdStanModel(stan_file=os.path.join(config_f["models_directory"], "model_parcialmente_agrupado.stan"))
model_parcialmente_agrupado_fit = model_parcialmente_agrupado.sample(
    data=radon_data, 
    show_progress=False, 
    chains=6,
    iter_warmup= 1000,
    iter_sampling=30000)

22:52:02 - cmdstanpy - INFO - compiling stan file /home/juanpalms/Desktop/MCD/Bayesiana/PROYECTO/modelacion-bayesiana-proyecto-final-bayes-radioactivo/modelos/model_parcialmente_agrupado.stan to exe file /home/juanpalms/Desktop/MCD/Bayesiana/PROYECTO/modelacion-bayesiana-proyecto-final-bayes-radioactivo/modelos/model_parcialmente_agrupado
22:52:16 - cmdstanpy - INFO - compiled model executable: /home/juanpalms/Desktop/MCD/Bayesiana/PROYECTO/modelacion-bayesiana-proyecto-final-bayes-radioactivo/modelos/model_parcialmente_agrupado
22:52:16 - cmdstanpy - INFO - CmdStan start processing
22:52:16 - cmdstanpy - INFO - Chain [1] start processing
22:52:16 - cmdstanpy - INFO - Chain [2] start processing
22:52:16 - cmdstanpy - INFO - Chain [3] start processing
22:52:16 - cmdstanpy - INFO - Chain [4] start processing
22:52:16 - cmdstanpy - INFO - Chain [5] start processing
22:52:16 - cmdstanpy - INFO - Chain [6] start processing
22:53:01 - cmdstanpy - INFO - Chain [6] done processing
22:53:01 - c

<ExecutionResult object at 7f75063e0050, execution_count=None error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f7505f008d0, raw_cell="model_parcialmente_agrupado = CmdStanModel(stan_fi.." store_history=False silent=False shell_futures=True cell_id=None> result=None>

### Analizando las cadenas

In [36]:
summary=model_parcialmente_agrupado_fit.summary().round(2)

In [38]:
(summary[1:86]["R_hat"]>=1).count()

85

In [40]:
parametros=model_parcialmente_agrupado_fit.draws_pd(vars=['alpha', 'beta', 'sigma'])

In [41]:
parametros

,alpha[1],alpha[2],alpha[3],alpha[4],alpha[5],alpha[6],alpha[7],alpha[8],alpha[9],alpha[10],...,alpha[78],alpha[79],alpha[80],alpha[81],alpha[82],alpha[83],alpha[84],alpha[85],beta,sigma
0,0.824548,0.820704,1.52162,1.36117,1.11932,1.543700,1.79375,1.66989,1.077750,1.59800,...,1.60659,1.277010,1.21058,2.52250,2.067340,1.51364,1.92950,1.669810,-0.773347,0.739571
1,1.431100,1.007030,1.18706,1.69966,1.49267,1.632950,1.82215,1.88965,1.253670,1.80236,...,1.32458,1.153310,1.42082,2.13578,1.091290,1.77000,1.57653,1.123410,-0.762985,0.770421
2,0.583700,0.867225,1.57354,1.42088,1.45303,1.562400,1.91190,1.48267,0.985453,1.16663,...,1.20061,0.984376,1.17838,1.88576,2.071060,1.45329,1.47220,1.493880,-0.610852,0.750177
3,1.502960,0.903858,1.16825,1.61730,1.52966,1.653470,1.58766,1.93964,0.786562,1.56386,...,1.49696,1.673510,1.17662,2.16985,1.721700,1.34943,1.29162,0.848085,-0.750055,0.763442
4,1.234680,1.008540,1.51091,1.54892,1.58615,1.470520,2.05129,1.70303,1.448980,1.60936,...,1.30624,0.589111,1.46450,1.43022,1.652670,1.71960,1.83429,1.614060,-0.681793,0.776874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179995,1.275190,0.861656,1.50406,1.33504,1.56439,0.897331,2.13607,1.24408,1.184670,1.46852,...,1.28949,1.212720,1.29235,1.69670,1.030310,1.84079,1.60674,1.552700,-0.752114,0.748644
179996,1.227830,0.986632,1.49844,1.87365,1.36444,1.834620,1.56906,2.15823,1.191900,1.58275,...,1.62068,1.157830,1.41075,2.14098,2.419500,1.27429,1.52753,1.274930,-0.696807,0.785801
179997,1.307780,0.907644,1.60783,1.42610,1.63932,1.397900,2.23262,1.34280,1.147320,1.56246,...,1.28212,1.055980,1.35717,1.84368,1.002270,1.90277,1.73249,1.551700,-0.686128,0.747674
179998,0.877886,0.828095,1.45717,1.63561,1.15313,1.766390,1.73354,2.16910,0.986915,1.51427,...,1.78552,1.069530,1.40314,2.05324,2.476450,1.55703,1.69252,1.205420,-0.681344,0.781389


In [42]:
summary

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
lp__,-258.98,0.04,9.23,-274.69,-258.67,-244.31,42269.9,167.04,1.0
mu_alpha,1.47,0.00,0.05,1.38,1.47,1.56,107782.0,425.94,1.0
sigma_alpha,0.34,0.00,0.05,0.27,0.34,0.43,71985.5,284.47,1.0
alpha[1],1.19,0.00,0.26,0.75,1.19,1.61,362977.0,1434.42,1.0
alpha[2],0.93,0.00,0.10,0.76,0.93,1.10,288643.0,1140.67,1.0
...,...,...,...,...,...,...,...,...,...
y_rep[923],1.59,0.00,0.79,0.30,1.59,2.89,186857.0,738.42,1.0
y_rep[924],1.59,0.00,0.79,0.30,1.60,2.88,185266.0,732.14,1.0
y_rep[925],1.59,0.00,0.79,0.30,1.59,2.89,184664.0,729.76,1.0
y_rep[926],1.39,0.00,0.82,0.04,1.39,2.74,189013.0,746.94,1.0


## 4 Modelo parcialmente agrupado: distintos interceptos y coeficientes por condado

Modelo parcialmente agrupado B

$ \large log\_radon_i  \sim N( \alpha_{j[i]} + \beta_{j[i]} Piso_i  , \sigma)$ <br>

$\large \alpha_j \sim N(\mu_{\alpha},\sigma_{\alpha})$ <br>
$\large \mu_\alpha \sim N(\mu,\sigma)$ <br>
$\large \sigma_\alpha \sim N(\mu,\sigma)$ <br>
$\large \beta_j \sim N(\mu_\beta,\sigma_\beta)$ <br>
$\large \mu_\beta \sim N(\mu,\sigma)$ <br>
$\large \sigma_\beta \sim N(\mu,\sigma)$ <br>
$\large \sigma \sim N(\mu,\sigma)$ <br>

$\large j \in \{1,2....85\}$

In [39]:
modelString="""
    data {
  int<lower=1> N;  // observations
  int<lower=1> J;  // counties
  array[N] int<lower=1, upper=J> county;
  vector[N] x;
  vector[N] y;
}
parameters {
  real mu_alpha;
  real<lower=0> sigma_alpha;
  vector<offset=mu_alpha, multiplier=sigma_alpha>[J] alpha;  // non-centered parameterization
  real mu_beta;
  real<lower=0> sigma_beta;
  vector<offset=mu_beta, multiplier=sigma_beta>[J] beta; // non-centered parameterization
  real<lower=0> sigma;
}
model {
  y ~ normal(alpha[county] + beta[county] .* x, sigma);
  alpha ~ normal(mu_alpha, sigma_alpha); // partial-pooling
  beta ~ normal(mu_beta, sigma_beta); // partial-pooling
  mu_alpha ~ normal(0, 10);
  sigma_alpha ~ normal(0, 10);
  mu_beta ~ normal(0, 10);
  sigma_beta ~ normal(0, 10);
  sigma ~ normal(0, 10);
}
generated quantities {
  array[N] real y_rep;
  
  for (i in 1:N) {
    y_rep[i] = normal_rng(alpha[county[i]] + beta[county[i]] * x[i], sigma);
  }
}


"""

In [40]:
# Creo el archivo de Stan
modelo_parcialmente_agrupadoB = os.path.join(config_f["models_directory"],"modelo_parcialmente_agrupadoB.stan")
with open(modelo_parcialmente_agrupadoB, 'w') as f:
    f.write(modelString)

In [43]:
%%log_run Modelo parcialmente agrupado B (jerarquico intercepto y variable explicativa)
modelo_parcialmente_agrupadoB= CmdStanModel(stan_file=os.path.join(config_f["models_directory"], "modelo_parcialmente_agrupadoB.stan"))
modelo_parcialmente_agrupadoB_fit = modelo_parcialmente_agrupadoB.sample(
    data=radon_data, 
    show_progress=False, 
    chains=4,
    iter_warmup= 1000,
    iter_sampling=30000)

23:04:45 - cmdstanpy - INFO - CmdStan start processing
23:04:45 - cmdstanpy - INFO - Chain [1] start processing
23:04:45 - cmdstanpy - INFO - Chain [2] start processing
23:04:45 - cmdstanpy - INFO - Chain [3] start processing
23:04:45 - cmdstanpy - INFO - Chain [4] start processing
23:05:22 - cmdstanpy - INFO - Chain [1] done processing
23:05:25 - cmdstanpy - INFO - Chain [4] done processing
23:05:36 - cmdstanpy - INFO - Chain [2] done processing
23:05:38 - cmdstanpy - INFO - Chain [3] done processing
23:05:38 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: offset_multiplier_constrain: multiplier is 0, but must be positive finite! (in '/home/juanpalms/Desktop/MCD/Bayesiana/PROYECTO/modelacion-bayesiana-proyecto-final-bayes-radioactivo/modelos/modelo_parcialmente_agrupadoB.stan', line 12, column 2 to column 59)
Exception: offset_multiplier_constrain: multiplier is 0, but must be positive finite! (in '/home/juanpalms/Desktop/MCD/Bayesiana/PROYECTO/modelacion-bayesiana

<ExecutionResult object at 7f7506422150, execution_count=None error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f7506422190, raw_cell="modelo_parcialmente_agrupadoB= CmdStanModel(stan_f.." store_history=False silent=False shell_futures=True cell_id=None> result=None>

In [47]:
summary=modelo_parcialmente_agrupadoB_fit.summary().round(2)

In [48]:
summary

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
lp__,-294.68,0.14,12.92,-316.36,-294.44,-273.75,8234.1,135.55,1.0
mu_alpha,1.47,0.00,0.05,1.38,1.47,1.55,15335.5,252.46,1.0
sigma_alpha,0.35,0.00,0.05,0.27,0.34,0.43,13981.3,230.16,1.0
alpha[1],1.18,0.00,0.26,0.74,1.18,1.60,51896.0,854.32,1.0
alpha[2],0.93,0.00,0.10,0.77,0.93,1.10,64514.5,1062.05,1.0
...,...,...,...,...,...,...,...,...,...
y_rep[923],1.60,0.00,0.78,0.32,1.60,2.88,40891.3,673.16,1.0
y_rep[924],1.59,0.00,0.78,0.30,1.59,2.88,41205.2,678.33,1.0
y_rep[925],1.59,0.00,0.78,0.30,1.60,2.88,39817.0,655.48,1.0
y_rep[926],1.38,0.00,0.81,0.03,1.38,2.72,41009.0,675.10,1.0


In [53]:
modelo_parcialmente_agrupadoB_pd = modelo_parcialmente_agrupadoB_fit.draws_pd(vars=['alpha', 'beta', 'sigma', 'mu_alpha','sigma_alpha','mu_beta','sigma_beta'])
print(f'sample draws shape:  {modelo_parcialmente_agrupadoB_pd.shape}')
modelo_parcialmente_agrupadoB_pd.head(3)

sample draws shape:  (40000, 175)


,alpha[1],alpha[2],alpha[3],alpha[4],alpha[5],alpha[6],alpha[7],alpha[8],alpha[9],alpha[10],...,beta[81],beta[82],beta[83],beta[84],beta[85],sigma,mu_alpha,sigma_alpha,mu_beta,sigma_beta
0,1.438060,1.162770,1.24724,1.46407,1.61009,1.30750,1.89312,1.58565,1.08868,1.46970,...,-0.581651,-0.926149,-1.28570,-0.788142,-1.200420,0.740894,1.44678,0.354049,-0.627878,0.439417
1,1.026870,0.709631,1.62438,1.71445,1.38339,1.89378,1.88952,1.38900,1.08031,1.59571,...,-0.709942,-0.966270,-1.20108,-1.083430,-0.307367,0.742750,1.52468,0.345192,-0.886530,0.289611
2,0.869253,1.042340,1.39504,1.75346,1.33042,1.55675,2.03649,1.72988,1.10132,1.47942,...,-0.377968,-0.626713,-1.01845,-0.492135,-0.733370,0.756102,1.50247,0.295745,-0.625666,0.286920


In [55]:
modelo_parcialmente_agrupadoB_pd = modelo_parcialmente_agrupadoB_fit.draws_pd(vars=['mu_alpha', 'sigma_alpha', 'mu_beta','sigma_beta'])
print(f'sample draws shape:  {modelo_parcialmente_agrupadoB_pd.shape}')
modelo_parcialmente_agrupadoB_pd.head(3)

sample draws shape:  (40000, 4)


,mu_alpha,sigma_alpha,mu_beta,sigma_beta
0,1.44678,0.354049,-0.627878,0.439417
1,1.52468,0.345192,-0.886530,0.289611
2,1.50247,0.295745,-0.625666,0.286920


Media de beta (dist. jerarquica): -0.67 <br>
Media de alpha (dist. jerarquica): 0.27